# Notebook 2: Working with CSV Data in Colab

**Session 2: The AI-Empowered Coder**  
*Generative AI for Scholarship — Harvard HDSI & FAS*

© 2026 President and Fellows of Harvard College. Licensed under CC BY-NC 4.0.

---

## What This Notebook Covers

This notebook demonstrates how to **create, save, read, and analyze CSV data**
using Google Drive as persistent storage.

1. **Mount** Google Drive so Colab can read and write files there
2. **Generate** a synthetic dataset (2 columns, 100 rows)
3. **Write** it to a CSV file in your Google Drive
4. **Read** the CSV file back into Python
5. **Analyze** the data with summary statistics and a plot
6. **Save** analysis results to a new CSV file in Drive

### Why Google Drive?

Colab runs on Google's servers, not your laptop. It has a temporary
working directory (`/content/`) that is **deleted** whenever the runtime
disconnects. If you save files there, they vanish when your session ends.

By saving to **Google Drive** instead, your files persist across sessions.
You can also access them from any browser, share them, and download them
to your laptop whenever you want.

In [ ]:
# ============================================================
# Run this cell first to increase font sizes for presentation.
# ============================================================

from IPython.display import HTML, display
display(HTML('''<style>
/* Larger fonts for presenting on a projector */
.text_cell_render, .markdown-cell-content, .rendered_html {
    font-size: 18px !important;
    line-height: 1.6 !important;
}
.CodeMirror pre, .monaco-editor .view-lines {
    font-size: 16px !important;
    line-height: 1.5 !important;
}
.output pre, .output_text pre, .output_area pre {
    font-size: 15px !important;
    line-height: 1.4 !important;
}
</style>'''))
print("Font sizes increased for presentation.")

---

## Step 1: Mount Google Drive

Before we can save files to Google Drive, we need to **mount** it.
This makes your entire Drive accessible as a folder inside Colab.

**When you run this cell:**
- Colab will ask you to **authorize access** to your Google Drive
- A pop-up window will appear asking you to sign in with your Google account
- Click **Allow** to grant access
- This authorization is needed once per Colab session

In [ ]:
# ============================================================
# Mount Google Drive.
#
# After this runs, your Drive files are accessible at:
#   /content/drive/MyDrive/
#
# A pop-up will ask you to authorize access — click Allow.
# ============================================================

from google.colab import drive

drive.mount('/content/drive')

print("\nGoogle Drive is now mounted.")
print("Your files are at: /content/drive/MyDrive/")

In [ ]:
# ============================================================
# Set the folder where we'll save our data files.
#
# We use the "Colab Notebooks" folder in your Google Drive,
# which is the default folder Colab creates for you.
# If it doesn't exist yet, we create it.
# ============================================================

import os

drive_folder = '/content/drive/MyDrive/Colab Notebooks'

# Create the folder if it doesn't already exist
os.makedirs(drive_folder, exist_ok=True)

print(f"Working folder: {drive_folder}")
print(f"Folder exists:  {os.path.isdir(drive_folder)}")

---

## Step 2: Import Libraries

We need two libraries, both pre-installed in Colab:
- **NumPy** — for generating random numerical data
- **Pandas** — for working with tabular data (DataFrames) and CSV files

In [ ]:
# ============================================================
# Import the libraries we need.
# Both are pre-installed in Google Colab.
# ============================================================

import numpy as np       # Numerical computing and random number generation
import pandas as pd      # DataFrames and CSV file I/O

print(f"NumPy version:  {np.__version__}")
print(f"Pandas version: {pd.__version__}")

---

## Step 3: Generate Synthetic Data

We'll create a dataset with **100 rows** and **2 columns**:

| Column | Description |
|--------|-------------|
| `day` | Day number (1 through 100) — represents consecutive days of observation |
| `temperature_C` | Daily average temperature in Celsius — a seasonal pattern with random noise |

The temperature follows a **sinusoidal seasonal pattern** (warmer in summer,
cooler in winter) plus random day-to-day variation. This mimics what you
might see in real weather data.

We use `np.random.seed(42)` to make the random numbers **reproducible** —
everyone running this notebook gets the same data.

In [ ]:
# ============================================================
# Generate a synthetic dataset: 100 days of temperature data.
#
# The temperature model is:
#   T(day) = 15 + 10 * sin(2π * day / 100) + noise
#
# This gives a baseline of 15°C, a seasonal swing of ±10°C,
# and random daily variation (noise) with std dev of 2°C.
# ============================================================

# Set random seed for reproducibility
np.random.seed(42)

# Number of data points
n_rows = 100

# Column 1: Day number (1 through 100)
day = np.arange(1, n_rows + 1)

# Column 2: Temperature with a seasonal pattern + random noise
seasonal_pattern = 15 + 10 * np.sin(2 * np.pi * day / n_rows)
daily_noise = np.random.normal(loc=0, scale=2, size=n_rows)
temperature_C = seasonal_pattern + daily_noise

# Round temperatures to 1 decimal place (realistic precision)
temperature_C = np.round(temperature_C, 1)

# Assemble into a Pandas DataFrame
df = pd.DataFrame({
    'day': day,
    'temperature_C': temperature_C
})

# Display the first 10 rows to verify the data looks reasonable
print(f"Generated {len(df)} rows with {len(df.columns)} columns.")
print(f"\nFirst 10 rows:")
df.head(10)

---

## Step 4: Write the Data to a CSV File on Google Drive

CSV (Comma-Separated Values) is the most common format for sharing tabular data.
Pandas makes it simple to write a DataFrame to CSV with `df.to_csv()`.

We save the file to our **Google Drive** folder so it persists across sessions.
You can verify the file exists by opening Google Drive in your browser
and looking inside the "Colab Notebooks" folder.

In [ ]:
# ============================================================
# Write the DataFrame to a CSV file on Google Drive.
#
# os.path.join() builds the full path correctly, handling
# the space in "Colab Notebooks" automatically.
#
# index=False prevents pandas from writing the row index
# as an extra column (we don't need it since 'day' already
# serves as our identifier).
# ============================================================

output_file = os.path.join(drive_folder, 'daily_temperatures.csv')

df.to_csv(output_file, index=False)

size_kb = os.path.getsize(output_file) / 1024
print(f"Data written to: {output_file}")
print(f"File size: {size_kb:.1f} KB")

### Verify: Look at the Raw CSV File

Let's peek at the first few lines of the file to see what CSV format looks like.
The `!head` command is a Linux shell command (Colab runs on Linux) that
shows the first N lines of a file.

In [ ]:
# ============================================================
# Display the first 5 lines of the CSV file.
# The '!' prefix runs a shell command instead of Python.
# Quotes around the path handle the space in "Colab Notebooks".
# ============================================================

!head -5 "/content/drive/MyDrive/Colab Notebooks/daily_temperatures.csv"

You should see something like:
```
day,temperature_C
1,16.0
2,15.7
3,16.3
4,18.0
```

The first line is the **header** (column names), and each subsequent line
is one row of data with values separated by commas.

---

## Step 5: Read the CSV File Back

Now let's read the file we just created, as if we were loading data
from an external source. This uses `pd.read_csv()`, which is the
most common way to load tabular data in Python.

In [ ]:
# ============================================================
# Read the CSV file from Google Drive into a new DataFrame.
#
# pd.read_csv() automatically detects:
#   - The header row (column names)
#   - Data types (integers, floats, strings)
#   - The delimiter (comma by default)
# ============================================================

csv_path = os.path.join(drive_folder, 'daily_temperatures.csv')
df_loaded = pd.read_csv(csv_path)

# Confirm it loaded correctly
print(f"Loaded {len(df_loaded)} rows and {len(df_loaded.columns)} columns.")
print(f"Column names: {list(df_loaded.columns)}")
print(f"Data types:")
print(df_loaded.dtypes)
print(f"\nFirst 5 rows:")
df_loaded.head()

---

## Step 6: Analyze the Data

Pandas provides built-in methods for **summary statistics**.
The `.describe()` method gives you count, mean, standard deviation,
min, max, and quartiles in one call.

In [ ]:
# ============================================================
# Summary statistics for the temperature column.
#
# .describe() returns: count, mean, std, min, 25%, 50%, 75%, max
# ============================================================

print("Summary Statistics")
print("=" * 30)
print(df_loaded['temperature_C'].describe())

print(f"\nAdditional statistics:")
print(f"  Median:   {df_loaded['temperature_C'].median():.1f} °C")
print(f"  Range:    {df_loaded['temperature_C'].min():.1f} – {df_loaded['temperature_C'].max():.1f} °C")
print(f"  Variance: {df_loaded['temperature_C'].var():.2f} °C²")

### Plot the Data

A plot makes the seasonal pattern immediately visible.
We'll add a horizontal line at the mean temperature for reference.

In [ ]:
# ============================================================
# Plot temperature vs. day to visualize the seasonal pattern.
# ============================================================

import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 4))

# Plot the temperature data as points connected by lines
ax.plot(df_loaded['day'], df_loaded['temperature_C'],
        color='#A51C30', linewidth=1, alpha=0.7, label='Daily temperature')

# Add a horizontal line at the mean
mean_temp = df_loaded['temperature_C'].mean()
ax.axhline(y=mean_temp, color='gray', linestyle='--', alpha=0.5,
           label=f'Mean ({mean_temp:.1f} °C)')

# Labels and formatting
ax.set_xlabel('Day')
ax.set_ylabel('Temperature (°C)')
ax.set_title('Daily Average Temperature — 100 Days of Synthetic Data')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## Step 7: Save Analysis Results to a New CSV

Let's add a computed column to the data and save the enhanced version
to a new CSV file on Google Drive. We'll convert Celsius to Fahrenheit
and include both.

In [ ]:
# ============================================================
# Add a Fahrenheit column and save the enhanced data
# to Google Drive.
#
# The conversion formula is: F = C × 9/5 + 32
# ============================================================

# Add the new column
df_loaded['temperature_F'] = np.round(df_loaded['temperature_C'] * 9/5 + 32, 1)

# Save to a new CSV file on Google Drive
enhanced_file = os.path.join(drive_folder, 'daily_temperatures_enhanced.csv')
df_loaded.to_csv(enhanced_file, index=False)

print(f"Enhanced data written to: {enhanced_file}")
print(f"\nFirst 5 rows of enhanced data:")
df_loaded.head()

### Your Files Are on Google Drive

Because we saved to Google Drive (not to Colab's temporary `/content/` directory),
these CSV files **persist** even after your Colab session ends. You can:

- **Find them** in Google Drive under My Drive → Colab Notebooks
- **Download them** to your laptop from Google Drive at any time
- **Open them** in the next Colab session without regenerating the data
- **Share them** with collaborators via Google Drive sharing

Let's verify both files are there:

In [ ]:
# ============================================================
# List the CSV files we created in the Colab Notebooks folder.
# ============================================================

print(f"CSV files in {drive_folder}:\n")
for f in sorted(os.listdir(drive_folder)):
    if f.endswith('.csv'):
        size_kb = os.path.getsize(os.path.join(drive_folder, f)) / 1024
        print(f"  {f}  ({size_kb:.1f} KB)")

---

## Try It: Use the Magic Wand

Now try using Colab's **magic wand** (cell-level AI) to extend this analysis.
Click on the empty code cell below, then click the magic wand icon and try
one of these prompts:

- **"Create a histogram of the temperature data with 15 bins"**
- **"Find the 5 warmest and 5 coldest days in the dataset"**
- **"Calculate a 7-day rolling average and plot it on top of the raw data"**

In [ ]:
# Use the magic wand to generate analysis code here


---

## Summary

| Operation | Code | Notes |
|-----------|------|-------|
| Mount Google Drive | `drive.mount('/content/drive')` | Needed once per session |
| Set working folder | `drive_folder = '/content/drive/MyDrive/Colab Notebooks'` | Files here persist across sessions |
| Write CSV | `df.to_csv(os.path.join(drive_folder, 'file.csv'), index=False)` | `index=False` avoids writing row numbers |
| Read CSV | `pd.read_csv(os.path.join(drive_folder, 'file.csv'))` | Auto-detects headers and data types |
| Statistics | `df['col'].describe()` | Count, mean, std, min, quartiles, max |
| Plot | `plt.plot(x, y)` | Use matplotlib for quick visualization |

**Key takeaway:** Always save data files to **Google Drive**, not to Colab's
temporary `/content/` directory. Files on Drive persist across sessions and
are accessible from any browser.

---

**Next:** Open **Notebook 3** to learn how to read files (like images) from your Google Drive.